### Window Functions
- Using Window Functions for Aggregations:

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

data = [("Alice", "Sales", 1000), ("Bob", "Sales", 1500), ("Alice", "HR", 1200), ("Bob", "HR", 800)]
columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)

windowSpec = Window.partitionBy("Department").orderBy(col("Salary").desc())
df = df.withColumn("rank", rank().over(windowSpec))
df.show()


+-----+----------+------+----+
| Name|Department|Salary|rank|
+-----+----------+------+----+
|Alice|        HR|  1200|   1|
|  Bob|        HR|   800|   2|
|  Bob|     Sales|  1500|   1|
|Alice|     Sales|  1000|   2|
+-----+----------+------+----+



### Joining DataFrames
- Inner Join:

In [0]:
data1 = [("Alice", 1, 22), ("Bob", 2, 25)]
columns1 = ["Name", "ID","Age"]
df1 = spark.createDataFrame(data1, columns1)

data2 = [(1, "HR"), (2, "Sales"), (2, "Sales1")]
columns2 = ["ID", "Department"]
df2 = spark.createDataFrame(data2, columns2)

joined_df = df1.join(df2, "ID")
joined_df.show()


+---+-----+---+----------+
| ID| Name|Age|Department|
+---+-----+---+----------+
|  1|Alice| 22|        HR|
|  2|  Bob| 25|     Sales|
|  2|  Bob| 25|    Sales1|
+---+-----+---+----------+



### User-Defined Functions (UDFs)
- Creating and Using UDFs:

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def multiply_by_two(x):
    return x * 2

data1 = [("Alice", 1, 22), ("Bob", 2, 25)]
columns1 = ["Name", "ID","Age"]
df1 = spark.createDataFrame(data1, columns1)

multiply_by_two_udf = udf(multiply_by_two, IntegerType())
df_udf_result = df1.withColumn("Age_Twice", multiply_by_two_udf(col("Age")))
df_udf_result.show()


+-----+---+---+---------+
| Name| ID|Age|Age_Twice|
+-----+---+---+---------+
|Alice|  1| 22|       44|
|  Bob|  2| 25|       50|
+-----+---+---+---------+



### Joining DataFrames in PySpark

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("JoiningDataFrames").getOrCreate()

# Sample data for employees DataFrame
employees_data = [
    (1, "Alice", 101, 70000),
    (2, "Bob", 102, 80000),
    (3, "Charlie", 103, 50000),
    (4, "David", 101, 60000),
    (5, "Eve", 104, 90000)
]
employees_columns = ["employee_id", "name", "department_id", "salary"]
employees_df = spark.createDataFrame(employees_data, employees_columns)

# Sample data for departments DataFrame
departments_data = [
    (101, "Sales", "New York"),
    (102, "HR", "San Francisco"),
    (103, "IT", "Chicago"),
    (105, "Marketing", "Boston")
]
departments_columns = ["department_id", "department_name", "location"]
departments_df = spark.createDataFrame(departments_data, departments_columns)
display(employees_df)
display(departments_df)


employee_id,name,department_id,salary
1,Alice,101,70000
2,Bob,102,80000
3,Charlie,103,50000
4,David,101,60000
5,Eve,104,90000


department_id,department_name,location
101,Sales,New York
102,HR,San Francisco
103,IT,Chicago
105,Marketing,Boston


In [0]:
inner_join_df = employees_df.join(departments_df, on="department_id", how="inner")
inner_join_df.show()

+-------------+-----------+-------+------+---------------+-------------+
|department_id|employee_id|   name|salary|department_name|     location|
+-------------+-----------+-------+------+---------------+-------------+
|          101|          1|  Alice| 70000|          Sales|     New York|
|          101|          4|  David| 60000|          Sales|     New York|
|          102|          2|    Bob| 80000|             HR|San Francisco|
|          103|          3|Charlie| 50000|             IT|      Chicago|
+-------------+-----------+-------+------+---------------+-------------+



In [0]:
left_join_df = employees_df.join(departments_df, on="department_id", how="left")
left_join_df.show()

+-------------+-----------+-------+------+---------------+-------------+
|department_id|employee_id|   name|salary|department_name|     location|
+-------------+-----------+-------+------+---------------+-------------+
|          101|          1|  Alice| 70000|          Sales|     New York|
|          102|          2|    Bob| 80000|             HR|San Francisco|
|          103|          3|Charlie| 50000|             IT|      Chicago|
|          101|          4|  David| 60000|          Sales|     New York|
|          104|          5|    Eve| 90000|           null|         null|
+-------------+-----------+-------+------+---------------+-------------+



In [0]:
right_join_df = employees_df.join(departments_df, on="department_id", how="right")
right_join_df.show()

+-------------+-----------+-------+------+---------------+-------------+
|department_id|employee_id|   name|salary|department_name|     location|
+-------------+-----------+-------+------+---------------+-------------+
|          101|          4|  David| 60000|          Sales|     New York|
|          101|          1|  Alice| 70000|          Sales|     New York|
|          102|          2|    Bob| 80000|             HR|San Francisco|
|          103|          3|Charlie| 50000|             IT|      Chicago|
|          105|       null|   null|  null|      Marketing|       Boston|
+-------------+-----------+-------+------+---------------+-------------+



In [0]:
full_outer_join_df = employees_df.join(departments_df, on="department_id", how="outer")
full_outer_join_df.show()

+-------------+-----------+-------+------+---------------+-------------+
|department_id|employee_id|   name|salary|department_name|     location|
+-------------+-----------+-------+------+---------------+-------------+
|          101|          1|  Alice| 70000|          Sales|     New York|
|          101|          4|  David| 60000|          Sales|     New York|
|          102|          2|    Bob| 80000|             HR|San Francisco|
|          103|          3|Charlie| 50000|             IT|      Chicago|
|          104|          5|    Eve| 90000|           null|         null|
|          105|       null|   null|  null|      Marketing|       Boston|
+-------------+-----------+-------+------+---------------+-------------+



### Merging DataFrames Using Delta Lake
- Merging in Delta Lake is used to perform **upsert operations**, which means updating existing records and inserting new ones in a Delta table. This is particularly useful for incremental data processing.

- Example Scenario: Let's assume we have a transactions Delta table and a DataFrame transactions_updates with new transaction records and updates to existing transactions.

In [0]:
# Sample data for transactions DataFrame
transactions_data = [
    (1, 101, 50.0, "2024-01-01"),
    (2, 102, None , "2024-01-02"), # use None to put null value in delta table
    (3, 103, 200.0, "2024-01-03")
]
transactions_columns = ["transaction_id", "product_id", "amount", "transaction_date"]
transactions_df = spark.createDataFrame(transactions_data, transactions_columns)

# Write to Delta Lake
transactions_df.write.format("delta").mode("overwrite").save("/dbfs/delta/transactions")

# Sample data for transactions_updates DataFrame
transactions_updates_data = [
    (2, 102, 175.0, "2024-01-02"),  # Update existing transaction
    (4, 104, 125.0, "2024-01-04")   # New transaction
]
transactions_updates_columns = ["transaction_id", "product_id", "amount", "transaction_date"]
transactions_updates_df = spark.createDataFrame(transactions_updates_data, transactions_updates_columns)
display(transactions_df)
display(transactions_updates_df)

transaction_id,product_id,amount,transaction_date
1,101,50.0,2024-01-01
2,102,null,2024-01-02
3,103,200.0,2024-01-03


transaction_id,product_id,amount,transaction_date
2,102,175.0,2024-01-02
4,104,125.0,2024-01-04


In [0]:
from delta.tables import DeltaTable

# Define the path to the Delta table
delta_table_path = "/dbfs/delta/transactions"

# Load the Delta table using DeltaTable API
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Get the table details using describeDetail method
details_df = delta_table.detail()
display(details_df)
# Show the details, including location and schema
details_df.select(["location", "lastmodified"]).show(truncate=False)

# Print the entire details DataFrame for more information
details_df.show(truncate=False)


format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,1ecfd1b0-f566-4136-9a85-354870f64f56,null,null,dbfs:/dbfs/delta/transactions,2024-07-17T18:17:54.178+0000,2024-07-17T22:57:19.000+0000,List(),3,4352,Map(),1,2,"List(appendOnly, invariants)",Map()


+-----------------------------+-------------------+
|location                     |lastmodified       |
+-----------------------------+-------------------+
|dbfs:/dbfs/delta/transactions|2024-07-17 22:57:19|
+-----------------------------+-------------------+

+------+------------------------------------+----+-----------+-----------------------------+-----------------------+-------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+----------+
|format|id                                  |name|description|location                     |createdAt              |lastModified       |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |statistics|
+------+------------------------------------+----+-----------+-----------------------------+-----------------------+-------------------+----------------+--------+-----------+----------+----------------+----------------+--------

- Performing Merge (Upsert) Using Delta Lake


In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, when
# Load the Delta table
delta_table = DeltaTable.forPath(spark, "/dbfs/delta/transactions")
print("Old delta table")
raw_df = spark.read.format("delta").load("/dbfs/delta/transactions")
raw_df.show()
additional_value = 10
# Perform the merge (upsert) operation
delta_table.alias("tgt").merge(
    transactions_updates_df.alias("src"),
    "tgt.transaction_id = src.transaction_id"
).whenMatchedUpdate(set={
    "tgt.product_id": "src.product_id",
    "tgt.amount": when(col("tgt.amount").isNull(), col("src.amount") + additional_value)
                    .otherwise(col("tgt.amount") + col("src.amount")),
    "tgt.transaction_date": "src.transaction_date"
}).whenNotMatchedInsert(values={
    "transaction_id": "src.transaction_id",
    "product_id": "src.product_id",
    "amount": "src.amount",
    "transaction_date": "src.transaction_date"
}).execute()

print("New delta table")
# Show the updated Delta table
updated_df = spark.read.format("delta").load("/dbfs/delta/transactions")
updated_df.show()

# "tgt.amount": col("tgt.amount") + col("src.amount"), # way to use the column value as a int


Old delta table
+--------------+----------+------+----------------+
|transaction_id|product_id|amount|transaction_date|
+--------------+----------+------+----------------+
|             1|       101|  50.0|      2024-01-01|
|             3|       103| 200.0|      2024-01-03|
|             2|       102| 185.0|      2024-01-02|
|             4|       104| 125.0|      2024-01-04|
+--------------+----------+------+----------------+

New delta table
+--------------+----------+------+----------------+
|transaction_id|product_id|amount|transaction_date|
+--------------+----------+------+----------------+
|             1|       101|  50.0|      2024-01-01|
|             3|       103| 200.0|      2024-01-03|
|             2|       102| 360.0|      2024-01-02|
|             4|       104| 250.0|      2024-01-04|
+--------------+----------+------+----------------+



In [0]:
display(updated_df)

transaction_id,product_id,amount,transaction_date
1,101,50.0,2024-01-01
3,103,200.0,2024-01-03
2,102,360.0,2024-01-02
4,104,250.0,2024-01-04
